# Proyecto II: De palabras a Vectores

En este proyecto vamos a implementar uno de los sistemas más utilizados par aconvertir palabras a vectores **word2vec**. En este proyecto la idea es que ustedes creén una base de datos de texto, la cúal utilizarán para entrenar la códigficación de estás palabras, y a partir de allí implementar ciertos sistemas. Par afacilitar el trabajo, en este notebook pueden encontrar unas funciones que fueron extraidas de esté [link](https://towardsdatascience.com/a-word2vec-implementation-using-numpy-and-python-d256cf0e5f28), y adaptadas para trabajar con el lenguaje español. Sientanse libres de utilizar estas funciones o de adaptarlas asu gusto. Las funciones toman como entrada un texto en formato plano, y al finalizar entregan el vector de palabras de contexto y el vector target que puede ser utilizado para entrenar el modelo word2vec. Los datos de entrenamiento se entregan en una lista que contiene el vector de salida (codificación one hot de una palabra) y el vector que contiene la suma de los vectores hot encoded de las palabras de contexto. Para escoger el tamaño de la palabra de contexto deben modificar el parámetro $C$.

Para este proyecto necesitan realizar lo siguiente:

1. Obtengan un corpus de texto plano en Español, lo suficientemente largo, que contenga información sobre diferentes países. Este texto lo pueden extraer de wikipedia.
2. Con la información de este corpus entrenen un modelo word2vec, aquí debe seleccionar un valor $N$ que representa la longitud del vector que representará las palabras. Este modelo es una red neuronal con una sola capa oculta de $N$ neuronas.
3. Una vez tengan el modelo identifiquen la relación entre algunos países (no todos) de su corpus y sus capitales.
4. Mediante esta relación encuentren las capitales de los demás países que incluyeron en el corpus. Para esto deben realizar una búsqueda de vectores cercanos a un punto en el espacio $\mathbb{R}^{\text{N}}$. Esta búsqueda la pueden realizar con *Locally Sensitive Hashing*.
4. Realice una visualización en 2D o 3D de los vectores correspondientes a los países y sus capitales. Indicando los que usarón de referencia para encontrar la relación entre país y capital, y aquellos para los cuales no utilizarón esa relación.
5. Seleccione 5 Palabras y para ellas calcule las 10 palabras más parecidas (sinonimos) en orden de similitud.
6. Construya un nuevo corpus, más pequeño, sobre un tema cualquiera. Para este corpus necesita la versión en Español y en Ingles de los textos. Una vez más pueden utilizar Wikipedia para esto. 
7. Obtenga las representaciones en vectores par alas palabras en Español y en Ingles.
8. De los datos obtenidos, escoga un conjunto $m$ de palabras en español, y las respectiva traducción en ingles (que se encuentren en el vocabulario que construyeron) y planteé el problema de transformacion $\mathcal{X}\mathbf{R} = \mathcal{Y}$. De tal forma que encuentren la matrix de "traducción" $\mathbf{R}$.
9. Realicé esta operación para varios valores de $m$.
10. Finalmente escoja algunas palabras en Español, apliqueles la transformación $\mathbf{R}$ y liste las 5 palabras más cercanas en $\mathcal{Y}$ a esta traducción.

Al finalizar los diferentes puntos del proyecto, contesten las isguientes preguntas:

1. ¿Qué problemas tuvierón a la hora de implementar el modelo word2vec?
2. ¿Cómo escogierón el parámetro $N$?
3. ¿Qué pueden decir de los resultados del modelo buscando la relación entre los países y las capitales?
4. ¿Qué tan bien funciona el modelo para los sinonimos?, ¿Cómo creen que se puede mejorar este modelo?
5. ¿Cómo afecta el parámetro $m$ el funcionamiento del modelo de traducción?
6. ¿Cómo mejorarian el modelo de traducción?
7. ¿Qué concluyen de este proyecto y de los resultados obtenidos?

Al finalizar deben entregar el notebook, con los archivos de soporte (los corpus) y demás elementos que consideren necesarios. Recuerde dar respuesta a las preguntas del notebook, y comentar cada parte del proceso. El proyecto se debe entregar a más tardar el **Domingo 29 de Marzo a las 12 de la noche**.


In [3]:
# Importacndo librerias necesarias

import re 
import numpy as np

In [4]:
# Función para leer y procesar el archivo en texto plano.

def get_file_data(fname, stop_word_removal='no'):
    file_contents = []
    with open(fname) as f:
        file_contents = f.read()
    text = []
    for val in file_contents.split('.'):
        val = re.sub(r'[,¡!¿?;-]+','.',val)
        val = re.sub(r'á','a',val)
        val = re.sub(r'é','e',val)
        val = re.sub(r'í','i',val)
        val = re.sub(r'ó','o',val)
        val = re.sub(r'ú','u',val)
        val = re.sub(r'Á','A',val)
        val = re.sub(r'É','E',val)
        val = re.sub(r'Í','I',val)
        val = re.sub(r'Ó','O',val)
        val = re.sub(r'Ú','U',val)
        val = re.sub(r'ñ','n',val)
        val = re.sub(r'Ñ','N',val)
        sent = re.findall("[A-Za-z]+", val)
        line = ''
        for words in sent:
            
            if stop_word_removal == 'yes': 
                if len(words) > 1 and words not in stop_words:
                    line = line + ' ' + words
            else:
                if len(words) > 1 :
                    line = line + ' ' + words
        text.append(line)
    return text

# Función para obtener un Vocabulario en función del texto procesado

def generate_dictinoary_data(text):
    word_to_index= dict()
    index_to_word = dict()
    corpus = []
    count = 0
    vocab_size = 0
    
    for row in text:
        for word in row.split():
            word = word.lower()
            corpus.append(word)
            if word_to_index.get(word) == None:
                word_to_index.update ( {word : count})
                index_to_word.update ( {count : word })
                count  += 1
    vocab_size = len(word_to_index)
    length_of_corpus = len(corpus)
    
    return word_to_index,index_to_word,corpus,vocab_size,length_of_corpus

# Función para generar representaciones one hot de los vectores target y del corpus

def get_one_hot_vectors(target_word,context_words,vocab_size,word_to_index):
    
    #Create an array of size = vocab_size filled with zeros
    trgt_word_vector = np.zeros(vocab_size)
    
    #Get the index of the target_word according to the dictionary word_to_index. 
    index_of_word_dictionary = word_to_index.get(target_word) 
    
    #Set the index to 1
    trgt_word_vector[index_of_word_dictionary] = 1
    
    #Repeat same steps for context_words but in a loop
    ctxt_word_vector = np.zeros(vocab_size)
    
    
    for word in context_words:
        index_of_word_dictionary = word_to_index.get(word) 
        ctxt_word_vector[index_of_word_dictionary] = 1
        
    return trgt_word_vector,ctxt_word_vector

# Función para generar los datos de entrenamiento para la red neuronal que representa el modelo word2vec

def generate_training_data(corpus,window_size,vocab_size,word_to_index,length_of_corpus,sample=None):

    training_data =  []
    training_sample_words =  []
    for i,word in enumerate(corpus):

        index_target_word = i
        target_word = word
        context_words = []

        #when target word is the first word
        if i == 0:  

            # trgt_word_index:(0), ctxt_word_index:(1,2)
            context_words = [corpus[x] for x in range(i + 1 , window_size + 1)] 


        #when target word is the last word
        elif i == len(corpus)-1:

            # trgt_word_index:(9), ctxt_word_index:(8,7), length_of_corpus = 10
            context_words = [corpus[x] for x in range(length_of_corpus - 2 ,length_of_corpus -2 - window_size  , -1 )]

        #When target word is the middle word
        else:

            #Before the middle target word
            before_target_word_index = index_target_word - 1
            for x in range(before_target_word_index, before_target_word_index - window_size , -1):
                if x >=0:
                    context_words.extend([corpus[x]])

            #After the middle target word
            after_target_word_index = index_target_word + 1
            for x in range(after_target_word_index, after_target_word_index + window_size):
                if x < len(corpus):
                    context_words.extend([corpus[x]])


        trgt_word_vector,ctxt_word_vector = get_one_hot_vectors(target_word,context_words,vocab_size,word_to_index)
        training_data.append([trgt_word_vector,ctxt_word_vector])   
        
        if sample is not None:
            training_sample_words.append([target_word,context_words])   
        
    return training_data,training_sample_words

In [5]:
# Ejecución del código

fname = 'paises' #Nombre del archivo
C = 2 # Número de palabras de contexto a la derecha y a la izquierda
text = get_file_data(fname, stop_word_removal='no')
word_to_index,index_to_word,corpus,vocab_size,length_of_corpus = generate_dictinoary_data(text)
training_data,training_sample_words = generate_training_data(corpus,C,vocab_size,word_to_index,length_of_corpus,sample=None)

# Solucion

In [14]:
vocab_size

1524

In [6]:
# TensorFlow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

from sklearn.model_selection import train_test_split

In [20]:
X = np.array([x[1] for x in training_data])
Y = np.array([y[0] for y in training_data])

In [21]:
# Dividiendo train test
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.3,random_state=42)

In [22]:
training_data[0]

[array([1., 0., 0., ..., 0., 0., 0.]), array([0., 1., 1., ..., 0., 0., 0.])]

In [29]:
model = Sequential()  
model.add(Dense(100,input_shape=(vocab_size,), activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='CategoricalCrossentropy', optimizer='rmsprop', metrics=['accuracy'])  

In [30]:
model.fit(X_train,Y_train,epochs=50,validation_data=(X_test,Y_test))

Epoch 1/50
103/103 [==============================] - 1s 9ms/step - loss: 6.9143 - accuracy: 0.0889 - val_loss: 6.4007 - val_accuracy: 0.0830
Epoch 2/50
103/103 [==============================] - 1s 7ms/step - loss: 5.9737 - accuracy: 0.0883 - val_loss: 6.3893 - val_accuracy: 0.0830
Epoch 3/50
103/103 [==============================] - 1s 7ms/step - loss: 5.7286 - accuracy: 0.0883 - val_loss: 6.4507 - val_accuracy: 0.0830
Epoch 4/50
103/103 [==============================] - 1s 7ms/step - loss: 5.5663 - accuracy: 0.0919 - val_loss: 6.4648 - val_accuracy: 0.0908
Epoch 5/50
103/103 [==============================] - 1s 7ms/step - loss: 5.4239 - accuracy: 0.1004 - val_loss: 6.5015 - val_accuracy: 0.0951
Epoch 6/50
103/103 [==============================] - 1s 7ms/step - loss: 5.2911 - accuracy: 0.1108 - val_loss: 6.5508 - val_accuracy: 0.0958
Epoch 7/50
103/103 [==============================] - 1s 7ms/step - loss: 5.1667 - accuracy: 0.1199 - val_loss: 6.5782 - val_accuracy: 0.1050
Epoch 